<a href="https://colab.research.google.com/github/LMuc-ux/mdmcmeta/blob/main/Fluent_control_to_elabFTW.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests
import json
import pandas as pd

# Define elabFTW API URL and credentials (replace with your actual values)
ELABFTW_URL = "your_elabftw_instance_url"  # e.g., "https://elabftw.yourdomain.com"
ELABFTW_TOKEN = "your_elabftw_api_token"

# Function to create an elabFTW experiment
def create_experiment(title, description):
    """
    Creates a new experiment in elabFTW.

    Args:
        title (str): Title of the experiment.
        description (str): Description of the experiment.

    Returns:
        int: The ID of the created experiment, or None if creation failed.
    """

    url = f"{ELABFTW_URL}/api/v1/experiments.json"
    headers = {
        "Authorization": f"Bearer {ELABFTW_TOKEN}",
        "Content-Type": "application/json",
    }
    payload = {"title": title, "description": description}

    try:
        response = requests.post(url, headers=headers, data=json.dumps(payload))
        response.raise_for_status()  # Raise an exception for bad status codes (4xx or 5xx)
        return response.json()["id"]
    except requests.exceptions.RequestException as e:
        print(f"Error creating experiment: {e}")
        return None

# Function to upload data to an elabFTW experiment
def upload_data_to_experiment(experiment_id, data, filename="tecan_data.json"):
    """
    Uploads data to an existing experiment in elabFTW.

    Args:
        experiment_id (int): The ID of the elabFTW experiment.
        data (dict or list): The data to upload.
        filename (str, optional): The filename to use in elabFTW. Defaults to "tecan_data.json".
    """

    url = f"{ELABFTW_URL}/api/v1/experiments/{experiment_id}/datablocks.json"
    headers = {
        "Authorization": f"Bearer {ELABFTW_TOKEN}",
    }
    payload = {"content": json.dumps(data), "filename": filename}

    try:
        response = requests.post(url, headers=headers, data=payload)
        response.raise_for_status()
        print(f"Data uploaded to experiment {experiment_id} successfully.")
    except requests.exceptions.RequestException as e:
        print(f"Error uploading data to experiment {experiment_id}: {e}")

# Load TECAN data from file
def load_tecan_data(filepath):
    """
    Loads TECAN data from a file.  Assumes a CSV-like structure based on your example.
    Adjust as needed for your actual TECAN output format.

    Args:
        filepath (str): Path to the TECAN data file.

    Returns:
        dict:  A dictionary representing the TECAN data.
    """

    try:
        df = pd.read_csv(filepath, sep=';')  # Adjust 'sep' if needed
        data = df.to_dict(orient='records')
        return data
    except FileNotFoundError:
        print(f"Error: File not found at {filepath}")
        return None
    except Exception as e:
        print(f"Error loading TECAN data: {e}")
        return None

if __name__ == '__main__':
    # 1. Load TECAN data
    tecan_data = load_tecan_data("VariabiliCelero.txt")  # Replace with your file path
    if tecan_data is None:
        exit()

    # 2. Create a new elabFTW experiment
    experiment_title = "TECAN Run - Celero Data"
    experiment_description = "Data from a TECAN run processing Celero samples."
    experiment_id = create_experiment(experiment_title, experiment_description)

    if experiment_id is None:
        exit()

    # 3. Upload the TECAN data to the experiment
    upload_data_to_experiment(experiment_id, tecan_data, "celero_data.json")
    #upload_data_to_experiment(experiment_id, tecan_data, "celero_data.csv")  #If you want csv
    #upload_data_to_experiment(experiment_id, tecan_data, "celero_data.txt")  #If you want txt

['.config', 'sample_data']
